In [1]:
import time
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn import metrics
from load import create_pipeline
import os
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
import itertools

In [2]:

# 전처리
# Get Dataset
files={
    'CICI':'/home/irteam/junghye-dcloud-dir/MLAC/data/encoded_ConcatedCICI.csv',
    'UNSW': '/home/irteam/junghye-dcloud-dir/MLAC/data/encoded_ConcatedUNSW.csv'
}

data = pd.read_csv(files['UNSW'])
data=data[np.isfinite(data).all(1)]
#target = data['attack_category']
binary_target=data['label']
#multiclass_labels_1=data['nist_category']
#multiclass_labels_2=data['attack_category']
#class_1_data=data[data['nist_category']==0]
#class_2_data=data[data['nist_category']==1]
#class_3_data=data[data['nist_category']==2]
#class_4_data=data[data['nist_category']==3]
data=data.drop(labels=['label','attack_category','nist_category','Unnamed: 0'],axis=1)

In [3]:
# Define Models
models = []
models.append(('RF', RandomForestClassifier(max_depth=5, n_estimators=5, max_features=3)))    
models.append(('CART', DecisionTreeClassifier(max_depth=5)))
models.append(('NB', GaussianNB()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('QDA', QuadraticDiscriminantAnalysis()))
models.append(('LR', LogisticRegression(solver='lbfgs', max_iter=200)))
models.append(('ABoost', AdaBoostClassifier()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('MLP', MLPClassifier()))

In [4]:
df=pd.DataFrame(columns=['name','acc','f1_mi','f1_ma','f1_we','recall_mi','recall_ma','recall_we']+\
                 ['precision_mi','precision_ma','precision_we'])
eval_path='/home/irteam/junghye-dcloud-dir/MLAC/evaluation'
confusion_path='/home/irteam/junghye-dcloud-dir/MLAC/confusion_matrix/UNSW_binary'
cnt=0

In [5]:
# train_test_split
X_train,X_test,y_train,y_test=train_test_split(data,binary_target,test_size=0.3, shuffle=True, stratify=binary_target, random_state=34)

In [6]:
def plot_confusion_matrix(con_mat,labels,title:str,cmap=plt.cm.get_cmap('Blues'),normalize=False):
    plt.imshow(con_mat,interpolation='nearest',cmap=cmap)
    plt.title(title)
    plt.colorbar()
    marks=np.arange(len(labels))
    nlabels=[]
    for k in range(len(con_mat)):
        n=sum(con_mat[k])
        nlabel='{0}(n={1})'.format(labels[k],n)
        nlabels.append(nlabel)

    plt.xticks(marks,labels)
    plt.yticks(marks,nlabels)

    thresh=con_mat.max()/2.
    if normalize:
        for i, j in itertools.product(range(con_mat.shape[0]), range(con_mat.shape[1])):
            plt.text(j, i, '{0}%'.format(con_mat[i, j] * 100 / n), horizontalalignment="center", color="white" if con_mat[i, j] > thresh else "black")
    else:
        for i, j in itertools.product(range(con_mat.shape[0]), range(con_mat.shape[1])):
            plt.text(j, i, con_mat[i, j], horizontalalignment="center", color="white" if con_mat[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    #plt.show()
    #이미지 저장
    plt.savefig(confusion_path+'/'+title+'.png',facecolor='#eeeeee')
    plt.clf()

In [ ]:
for name, model in models:
    #data_loader()
    # binary classification
    binary_model=create_pipeline(model)
   
    print('training start...'+name)
    binary_model.fit(X_train,y_train)
    
    #evaluation
    print('evaluation start...')
    binary_pred=binary_model.predict(X_test)
    #evaluation result
    model_eval=[]
    model_eval.append(name)
    
    acc = accuracy_score(y_test, binary_pred)
    f1_mi = f1_score(y_test, binary_pred,average='micro')
    f1_ma = f1_score(y_test, binary_pred,average='macro')
    f1_we = f1_score(y_test, binary_pred,average='weighted')
    recall_mi = recall_score(y_test, binary_pred, average='micro')
    recall_ma = recall_score(y_test, binary_pred, average='macro')
    recall_we = recall_score(y_test, binary_pred, average='weighted')
    precision_mi = precision_score(y_test, binary_pred, average='micro')
    precision_ma = precision_score(y_test, binary_pred, average='macro')
    precision_we = precision_score(y_test, binary_pred, average='weighted')
    
    model_eval.append(acc)
    model_eval.append(f1_mi)
    model_eval.append(f1_ma)
    model_eval.append(f1_we)
    model_eval.append(recall_mi)
    model_eval.append(recall_ma)
    model_eval.append(recall_we)
    model_eval.append(precision_mi)
    model_eval.append(precision_ma)
    model_eval.append(precision_we)
    
    #confusion_metrics
    confusion=metrics.confusion_matrix(y_test,binary_pred)
    plot_confusion_matrix(confusion,labels=list(set(binary_target)),title=name)
       


    print(f'name:{name},acc:{acc},f1_score:{f1_mi},{f1_ma},{f1_we},recall:{recall_mi},{recall_ma},{recall_we},precision:{precision_mi},{precision_ma},{precision_we}')
    df.loc[cnt]=model_eval

    cnt+=1
    

df.to_csv(os.path.join(eval_path,'UNSW_binary_eval.csv'),index=False)

training start...RF
evaluation start...
name:RF,acc:0.8747745643997114,f1_score:0.8747745643997114,0.4741527005681852,0.8173066233947383,recall:0.8747745643997114,0.5038162585018626,0.8747745643997114,precision:0.8747745643997114,0.9325883348983364,0.8892801845519689
training start...CART
evaluation start...
name:CART,acc:0.9875880138843263,f1_score:0.9875880138843263,0.9727214718805359,0.9877773908314246,recall:0.9875880138843263,0.9877987206925407,0.9875880138843263,precision:0.9875880138843263,0.9589042991111454,0.9883282160029505
training start...NB
evaluation start...
name:NB,acc:0.8888185261558068,f1_score:0.8888185261558068,0.5763495856185745,0.8483684766135416,recall:0.8888185261558068,0.5594441312263744,0.8888185261558068,precision:0.8888185261558068,0.9434758626275461,0.9013477015772378
training start...LDA
evaluation start...
name:LDA,acc:0.9874085889149671,f1_score:0.9874085889149671,0.9725653613898734,0.9876524560297925,recall:0.9874085889149671,0.9919363853594251,0.987408

In [1]:
from PIL import Image
import os
confusion_path='/home/irteam/junghye-dcloud-dir/MLAC/confusion_matrix/UNSW_binary'
#파일명
img_files=os.listdir(confusion_path)

images=[] # 이미지 리스트
for img_file in img_files:
    img=Image.open(os.path.join(confusion_path,img_file))
    img.resize((500,300))
    images.append(img)

# 크기
width,height=images[0].size
new_width=width*3
new_height=height*3
new_img=Image.new('RGB',(new_width,new_height))

#이미지 합치기
x_offset=0
y_offset=0
for img in images:
    new_img.paste(img,(x_offset,y_offset))
    x_offset+=width
    if x_offset==new_width:
        x_offset=0
        y_offset+=height


new_img.save(confusion_path+'/'+'UNSW_binary_merged.jpg')